# Uses of Recommendation System

- Rank Products by User Preference
- Product Similarity
    - Useful where we dont know much about a perticular user yet
- User Similarity
 

In [15]:
import numpy as np
import pandas as pd
import matrix_factorization_utilities as mfu

In [46]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

ratings = ratings.sample(n=500, random_state=0)

In [47]:
ratings.head()

,userId,movieId,rating,timestamp
875680,5843,2822,3.0,1111094142
1046906,7104,1320,3.5,1158923265
646861,4311,7027,4.0,1106430461
704385,4674,1968,4.0,949208685
798051,5329,4020,4.0,1173472330


In [48]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Convert the running list of user ratings into matrix using pivot table function

In [49]:
ratings_df = pd.pivot_table(ratings.drop('timestamp', axis=1),index="userId",columns="movieId",aggfunc=np.max)

In [50]:
ratings_df.head()

rating                                                                 \
movieId 1      6      10     17     21     22     25     31     32     34       
userId                                                                          
24         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
65         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
82         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
92         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
96         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ...                                                                 \
movieId  ... 88954  89492  91325  92439  94959  99114  100498 102481 102903   
userId   ...                                                                  
24       ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
65       ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
82       ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
92       ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
96       ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 116823  
userId          
24         NaN  
65         NaN  
82         NaN  
92         NaN  
96         NaN  

[5 rows x 431 columns]

## Factoring the above matrix in U and M 

In [51]:
U,M = mfu.low_rank_matrix_factorization(ratings_df.values,
                                       num_features=15,
                                       regularization_amount=0.1)

Optimization terminated successfully.
         Current function value: 151.365679
         Iterations: 508
         Function evaluations: 771
         Gradient evaluations: 771


In [52]:
ratings_df.values.shape

(455, 431)

In [53]:
#Find all predicted ratings by multiplying the U and M
predicted_ratings = np.matmul(U,M)

In [56]:
predicted_ratings_df = pd.DataFrame(index=ratings_df.index,
                                   columns=ratings_df.columns,
                                   data=predicted_ratings)

In [57]:
predicted_ratings_df

rating                                                              \
movieId    1         6         10        17        21        22        25       
userId                                                                          
24       0.313665 -1.082050  0.540763  0.938382  0.165207 -0.016310 -0.257712   
65      -0.964297  1.125327 -0.564699  0.465498 -1.976183  0.214988  0.946095   
82      -1.326826  0.904150 -0.563368 -0.002266  0.094598 -0.102808  1.142671   
92      -0.242554 -1.195928  1.655141  2.956317  0.485803  1.513837  1.270587   
96       2.156819 -1.014098  2.191980  0.022705  0.970406  0.752204  1.061324   
...           ...       ...       ...       ...       ...       ...       ...   
7051     0.446943  0.819400 -0.014253  0.658739  0.351055 -0.935333 -1.715098   
7056     2.074455  0.080378  1.962362 -0.181660  1.125733 -0.081422  0.029120   
7066    -0.059001 -1.361006 -1.566022 -1.034307 -0.325684 -0.919056  0.324753   
7091     1.799061  0.625004  1.794653  0.605489 -1.156759  0.805781  0.208632   
7104     1.625802  0.388030  1.084010  1.049081  0.201097  0.059272  0.101682   

                                       ...                                \
movieId    31        32        34      ...    88954     89492     91325    
userId                                 ...                                 
24      -0.014994  0.514799  0.459404  ...  0.991850  0.305045 -1.016901   
65       1.367824 -0.764456 -1.807716  ... -0.309371 -1.499193  0.326728   
82      -0.130513  0.141063 -0.085548  ... -1.208116  1.310026  1.032803   
92      -0.268454 -0.244086 -0.250334  ...  0.986558  0.734124 -1.554468   
96       0.115651 -0.523053 -0.232355  ... -0.227033 -0.712486 -0.753674   
...           ...       ...       ...  ...       ...       ...       ...   
7051    -0.427114  0.293732 -0.138816  ...  0.060254  1.122537 -0.107475   
7056    -0.244691  0.310401 -0.653329  ... -0.012247 -0.580355 -0.133694   
7066    -0.270302  0.177118 -0.118870  ... -0.770903  0.595415  1.144519   
7091     1.090722  0.916272 -0.408874  ...  0.573233 -0.188507  0.235926   
7104     0.252154  0.516599  0.537687  ...  0.792030  1.370720  0.386672   

                                                                               
movieId    92439     94959     99114     100498    102481    102903    116823  
userId                                                                         
24       0.039887  0.764007  0.039888  0.227829 -0.547468 -0.944462 -0.631413  
65      -0.490996  0.538059 -0.490996 -1.574131  0.491392  0.480361  0.000730  
82       0.031303 -0.914488  0.031301 -0.043056  0.449941  0.523752 -0.077026  
92       1.075955 -0.073128  1.075954  1.141964 -0.054641 -1.764060 -1.305980  
96       0.928397 -0.025492  0.928396  0.455849  2.223369  0.636239 -0.606858  
...           ...       ...       ...       ...       ...       ...       ...  
7051    -1.364165  0.923345 -1.364164 -0.364294 -1.024022 -0.676014 -0.321966  
7056     0.063938  0.484236  0.063938  1.019181  1.321986  0.892262 -0.686055  
7066     0.345880 -0.824246  0.345880 -0.015848 -0.753639  0.561284  1.900000  
7091     0.749942  2.870672  0.749944 -0.631878  1.007879 -0.700810 -0.914816  
7104    -0.155541  1.721419 -0.155539  0.396422 -0.910550 -1.546832 -0.759780  

[455 rows x 431 columns]